# Демонстрация работы классификатора

Сейчас мы находимся в папке <code>./source<code>, все данные находятся в папке <code>./data<code>. Помимо прочего, там содержатся txt-файлы распарсенных статей и файл с отметками некоторых статей. Перед тем, как обучать на них классификатор, полезно превратить эти статьи в мешки слов, причем постараться избавиться от словоформ.

За это отвественнен модуль <code>normalizing<code>. Он берет множество распарсенных статей, множество статей с отметками и "нормализует" их, то есть превращает в мешок слов те статьи, которые еще не нормализованы.

In [4]:
import normalizing

normalizing.normalizing()

Post 273408 normalized
Post 272936 normalized
Post 272922 normalized
Post 273992 normalized
Post 272948 normalized


Сейчас программа обработала только пять статей, остальные были обработаны ранее.

Для примера, вот так выглядит мешок слов для статьи <link>https://geektimes.ru/company/payonline/blog/272648/<link>:

In [5]:
!cat ../data/normalized_text/272648.txt

конец
февраль
visa
представить
необычный
платёжный
решение
приложение
который
помочь
водитель
оплачивать
бензин
заправка
выходить
машина
visa
honda
объединиться
чтобы
воплотить
жизнь
новое
платёжный
приложение
автомобиль
презентация
который
состояться
mobile
world
congress
барселона
позволять
оплачивать
парковка
заправка
автомобиль
выходить
план
разработка
подобный
приложение
автомобиль
visa
заявить
назад
visa
honda
создать
приложение
который
можно
пользоваться
непосредственно
панель
управление
автомобиль
сообщать
водитель
низок
уровень
топливо
строить
маршрут
ближний
заправка
когда
машина
парковаться
возле
топливный
насос
приложение
рассчитывать
стоимость
заправка
предлагать
совершить
оплата
прямо
через
панель
управление
visa
предполагать
дать
технология
есть
огромный
потенциал
через
четыре
около
миллиард
автомобиль
быть
подключить

Теперь приступим к собственно обучению.

In [1]:
from classifier import *

Создадим выборку для обучения и тренировки:

In [20]:
full_struct = get_a_structures(test_part=0.3)
train, test = full_struct.train, full_struct.test
print(len(train.data), len(test.data))

139 61


Для тестовой выборки отбиралось по 0.3 от всех элементов каждого класса. Если посмотреть на цифры, то будет следующее:

In [19]:
print("train positive:", train.target.count(1))
print("train negative:", train.target.count(-1))
print("test positive:", test.target.count(1))
print("test negative:", test.target.count(-1))

train positive: 60
train negative: 79
test positive: 26
test negative: 35


"Плохих" статей несколько больше. Но что уж тут поделать.

Для удобства создадим <code>Papeline<code>:

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='log'))
                    ])

<code>CountVectorizer<code> с помощью своего внутреннего словаря заодно отсеит всякие мусорные слова английского языка, что представляется довольно удобным. В качестве классификатора использует линейную регрессию с стохастическим градиентным спуском. Выборка для обучения разбивается на 5 частей.

Некоторые остальные параметры будет подбирать с помощью Gread Search-а. В качестве метрики для выбора наилучших параметров (<code>scorer<code>) будет auc-score.

In [5]:
from sklearn.grid_search import GridSearchCV
from sklearn import metrics

def scorer(estimator, X, Y):
    metric = metrics.roc_auc_score
    return metric(Y, estimator.predict_proba(X)[:, 1])

parameters = {'clf__penalty': ['l2', 'l1', 'elasticnet'],
              'clf__l1_ratio': [0.0, 0.01, 0.05],
              'clf__alpha': [0.001, 0.0001],
              'tfidf__use_idf': (True, False)
             }

searcher = GridSearchCV(estimator=text_clf,
                        param_grid=parameters,
                        scoring=scorer,
                        cv=5,
                        n_jobs=-1
                        )

И, собственно, запустим тренироваться на наших данных...

In [21]:
from time import time

print('Training... ', end='')
t = time()
searcher.fit(train.data, train.target)
print(round(time() - t, 3), "sec")

Training... 11.563 sec


Интересно узнать, каких результатов добилась наша сетка:

In [22]:
print("best score (auc):", round(searcher.best_score_, 3))
print("best parameters:", searcher.best_params_, sep='\n')    

best score (auc): 0.843
best parameters:
{'tfidf__use_idf': True, 'clf__penalty': 'l2', 'clf__alpha': 0.0001, 'clf__l1_ratio': 0.05}


Теперь самое время проверить результат на тестовой выборке!

In [23]:
best_cls = searcher.best_estimator_
predicted = best_cls.predict(test.data)
print("auc:", round(scorer(best_cls, test.data, test.target), 3))
print("mean:", round(np.mean(predicted == test.target), 3))
print(metrics.classification_report(test.target, predicted,
                                    target_names=test.target_names))

auc: 0.636
mean: 0.639
             precision    recall  f1-score   support

   Negative       0.67      0.74      0.70        35
   Positive       0.59      0.50      0.54        26

avg / total       0.63      0.64      0.63        61



Также мне очень нравится следующая матрица: она показывает в абсолютной величне, сколько было верно угаданных элементов класса, а также сколько было ошибок первого и второго рода.

(1, 1) --- верно угаданные элементы из класса -1.

(2, 2) --- верно угаданные элементы из класса 1.

(1, 2) и (2, 1) --- ошибки первого и второго рода.

Глядя на эту матрицу, становится ясно, что классификатор очень часто считает, что статья "плохая". Это может быть связано с тем, что этих примеров у нас больше в выборке.

In [24]:
print(metrics.confusion_matrix(test.target, predicted))

[[26  9]
 [13 13]]


В целом, вот результат. Он, на самом деле, не то чтобы очень хороший... Предположительно, стоит использовать какую-нибудь другую метрику, или накидать больше "хороших" статей.